In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

In [ ]:
os.chdir('drive/MyDrive')
os.listdir('src')

['models', '__pycache__', 'model_trainer.py', 'utils.py', 'transformations.py']

In [ ]:
from src.models.pretrained_models import VGG16Pretrained
import torch
from src.utils import load_data, load_data_augmented, load_data_augmented_pretrained
from src.model_trainer import ModelTrainer
from torchsummary import summary
from src.transformations import pretrained_transform, random_rotation_transform, horizontal_flip_transform_resized, color_jitter_transform_resized, combined_augmentation_transform_resized, cutout_transform_resized
torch.manual_seed(123)
from src.model_trainer import ModelTrainer
import itertools

In [ ]:
device_str = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(device_str)
print(device)
torch.set_num_threads(2)

cuda


In [ ]:
# cell for donwloading data

import os
import tarfile
import urllib.request
import shutil

download_dir = "/data"

if not os.path.exists(download_dir):
    os.makedirs(download_dir)

tar_path = os.path.join(download_dir, "CINIC-10.tar.gz")
cinic10_url = "https://datashare.is.ed.ac.uk/bitstream/handle/10283/3192/CINIC-10.tar.gz"
urllib.request.urlretrieve(cinic10_url, tar_path)
print("Finished downloading")

with tarfile.open(tar_path, "r:gz") as tar:
    tar.extractall(path=download_dir)

extracted_dir = os.path.join(download_dir, "CINIC-10")

if os.path.exists(extracted_dir) and os.path.isdir(extracted_dir):
    for item in os.listdir(extracted_dir):
        src_path = os.path.join(extracted_dir, item)
        dst_path = os.path.join(download_dir, item)
        shutil.move(src_path, dst_path)

print(download_dir)

Finished downloading
/data


In [ ]:
test_loader = load_data('/data/test', batch_size=128, shuffle=True, transform=pretrained_transform(), num_workers=1)
train_loader = load_data('/data/train', batch_size=128, shuffle=True, transform=pretrained_transform(), num_workers=1)
valid_loader = load_data('/data/valid', batch_size=128, shuffle=True, transform=pretrained_transform(), num_workers=1)

In [ ]:
# test_loader = load_data('./data/test', batch_size=128, shuffle=True, transform=normalized_simple_transform(), num_workers=1)
# train_loader = load_data('./data/train', batch_size=128, shuffle=True, transform=normalized_simple_transform(), num_workers=1)
# valid_loader = load_data('./data/valid', batch_size=512, shuffle=True, transform=normalized_simple_transform(), num_workers=1)

In [ ]:
training_process_grid = {
    "optimizer_type": ["adam", "sgd"],
    "learning_rate": [0.01, 0.001, 0.0001],
}

regularization_grid = {
    "weight_decay": [0.0001, 0.001, 0.01],
    "dropout": [0.2, 0.5, 0.7],
}

# grid search for training parameters with set L1 regularizer = 0 & dropout = 0
# grid search for regularization parameters with set optimizer = adam & learning_rate = 0.001

training_param_combinations = list(itertools.product(*training_process_grid.values()))
regularization_param_combinations = list(
    itertools.product(*regularization_grid.values())
)

In [ ]:
regularization_param_combinations

[(0.0001, 0.2),
 (0.0001, 0.5),
 (0.0001, 0.7),
 (0.001, 0.2),
 (0.001, 0.5),
 (0.001, 0.7),
 (0.01, 0.2),
 (0.01, 0.5),
 (0.01, 0.7)]

Test for learning parameters

In [ ]:
torch.manual_seed(123)
for optimizer_type, learning_rate in training_param_combinations:
    print(f"Training optimizer type {optimizer_type} with lr {learning_rate}")
    model = VGG16Pretrained(dropout=0)
    learning_rate_str = f"{learning_rate}".replace("0.", "")
    trainer = ModelTrainer(
        model=model,
        train_loader=train_loader,
        device=device,
        weight_decay=0,
        optimizer_type=optimizer_type,
        learning_rate=learning_rate,
        log_file=f"vgg_pretrained_{optimizer_type}_{learning_rate}.json",
        save_dir=f"./saved_models/vgg_pretrained/training_params_{optimizer_type}_{learning_rate_str}",
        max_batches=None,
        valid_loader=valid_loader
    )
    trainer.train(10)

In [ ]:
torch.manual_seed(123)
for weight_decay, dropout in regularization_param_combinations:
    model = VGG16Pretrained(dropout=dropout)
    w_str = f"{weight_decay}".replace("0.", "")
    d_str = f"{dropout}".replace("0.", "")
    trainer = ModelTrainer(
        model=model,
        train_loader=train_loader,
        device=device,
        optimizer_type="adam", # to fill
        learning_rate=0.0001, # to fill
        weight_decay=weight_decay,
        log_file=f"vgg_pretrained_{w_str}_{d_str}.json",
        save_dir=f"./saved_models/vgg_pretrained/regularization_params_{w_str}_{d_str}",
        max_batches=None,
        valid_loader=valid_loader
    )
    trainer.train(10)

In [ ]:
torch.manual_seed(123)
for weight_decay, dropout in regularization_param_combinations:
    model = VGG16Pretrained(dropout=dropout)
    w_str = f"{weight_decay}".replace("0.", "")
    d_str = f"{dropout}".replace("0.", "")
    trainer = ModelTrainer(
        model=model,
        train_loader=train_loader,
        device=device,
        optimizer_type="sgd", # to fill
        learning_rate=0.001, # to fill
        weight_decay=weight_decay,
        log_file=f"vgg_pretrained_{w_str}_{d_str}.json",
        save_dir=f"./saved_models/vgg_pretrained/sgd/regularization_params_{w_str}_{d_str}",
        max_batches=None,
        valid_loader=valid_loader
    )
    trainer.train(10)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:06<00:00, 82.1MB/s]


is cuda True
Batch count 0
Batch count 10
Batch count 20
Batch count 30
Batch count 40
Batch count 50
Batch count 60
Batch count 70
Batch count 80
Batch count 90
Batch count 100
Batch count 110
Batch count 120
Batch count 130
Batch count 140
Batch count 150
Batch count 160
Batch count 170
Batch count 180
Batch count 190
Batch count 200
Batch count 210
Batch count 220
Batch count 230
Batch count 240
Batch count 250
Batch count 260
Batch count 270
Batch count 280
Batch count 290
Batch count 300
Batch count 310
Batch count 320
Batch count 330
Batch count 340
Batch count 350
Batch count 360
Batch count 370
Batch count 380
Batch count 390
Batch count 400
Batch count 410
Batch count 420
Batch count 430
Batch count 440
Batch count 450
Batch count 460
Batch count 470
Batch count 480
Batch count 490
Batch count 500
Batch count 510
Batch count 520
Batch count 530
Batch count 540
Batch count 550
Batch count 560
Batch count 570
Batch count 580
Batch count 590
Batch count 600
Batch count 610
Batch 

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


is cuda True
Batch count 0
Batch count 10
Batch count 20
Batch count 30
Batch count 40
Batch count 50
Batch count 60
Batch count 70
Batch count 80
Batch count 90
Batch count 100
Batch count 110
Batch count 120
Batch count 130
Batch count 140
Batch count 150
Batch count 160
Batch count 170
Batch count 180
Batch count 190
Batch count 200
Batch count 210
Batch count 220
Batch count 230
Batch count 240
Batch count 250
Batch count 260
Batch count 270
Batch count 280
Batch count 290
Batch count 300
Batch count 310
Batch count 320
Batch count 330
Batch count 340
Batch count 350
Batch count 360
Batch count 370
Batch count 380
Batch count 390
Batch count 400
Batch count 410
Batch count 420
Batch count 430
Batch count 440
Batch count 450
Batch count 460
Batch count 470
Batch count 480
Batch count 490
Batch count 500
Batch count 510
Batch count 520
Batch count 530
Batch count 540
Batch count 550
Batch count 560
Batch count 570
Batch count 580
Batch count 590
Batch count 600
Batch count 610
Batch 

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


...

is cuda True
Batch count 0
Batch count 10
Batch count 20
Batch count 30
Batch count 40
Batch count 50
Batch count 60
Batch count 70
Batch count 80
Batch count 90
Batch count 100
Batch count 110
Batch count 120
Batch count 130
Batch count 140
Batch count 150
Batch count 160
Batch count 170
Batch count 180
Batch count 190
Batch count 200
Batch count 210
Batch count 220
Batch count 230
Batch count 240
Batch count 250
Batch count 260
Batch count 270
Batch count 280
Batch count 290
Batch count 300
Batch count 310
Batch count 320
Batch count 330
Batch count 340
Batch count 350
Batch count 360
Batch count 370
Batch count 380
Batch count 390
Batch count 400
Batch count 410
Batch count 420
Batch count 430
Batch count 440
Batch count 450
Batch count 460
Batch count 470
Batch count 480
Batch count 490
Batch count 500
Batch count 510
Batch count 520
Batch count 530
Batch count 540
Batch count 550
Batch count 560
Batch count 570
Batch count 580
Batch count 590
Batch count 600
Batch count 610
Batch 

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


is cuda True
Batch count 0
Batch count 10
Batch count 20
Batch count 30
Batch count 40
Batch count 50
Batch count 60
Batch count 70
Batch count 80
Batch count 90
Batch count 100
Batch count 110
Batch count 120
Batch count 130
Batch count 140
Batch count 150
Batch count 160
Batch count 170
Batch count 180
Batch count 190
Batch count 200
Batch count 210
Batch count 220
Batch count 230
Batch count 240
Batch count 250
Batch count 260
Batch count 270
Batch count 280
Batch count 290
Batch count 300
Batch count 310
Batch count 320
Batch count 330
Batch count 340
Batch count 350
Batch count 360
Batch count 370
Batch count 380
Batch count 390
Batch count 400
Batch count 410
Batch count 420
Batch count 430
Batch count 440
Batch count 450
Batch count 460
Batch count 470
Batch count 480
Batch count 490
Batch count 500
Batch count 510
Batch count 520
Batch count 530
Batch count 540
Batch count 550
Batch count 560
Batch count 570
Batch count 580
Batch count 590
Batch count 600
Batch count 610
Batch 

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


is cuda True
Batch count 0
Batch count 10
Batch count 20
Batch count 30
Batch count 40
Batch count 50
Batch count 60
Batch count 70
Batch count 80
Batch count 90
Batch count 100
Batch count 110
Batch count 120
Batch count 130
Batch count 140
Batch count 150
Batch count 160
Batch count 170
Batch count 180
Batch count 190
Batch count 200
Batch count 210
Batch count 220
Batch count 230
Batch count 240
Batch count 250
Batch count 260
Batch count 270
Batch count 280
Batch count 290
Batch count 300
Batch count 310
Batch count 320
Batch count 330
Batch count 340
Batch count 350
Batch count 360
Batch count 370
Batch count 380
Batch count 390
Batch count 400
Batch count 410
Batch count 420
Batch count 430
Batch count 440
Batch count 450
Batch count 460
Batch count 470
Batch count 480
Batch count 490
Batch count 500
Batch count 510
Batch count 520
Batch count 530
Batch count 540
Batch count 550
Batch count 560
Batch count 570
Batch count 580
Batch count 590
Batch count 600
Batch count 610
Batch 

# test for data augmentation

In [ ]:
dropout = 0.2
learning_rate=0.001
weight_decay=0.001
optimizer_type="sgd"

In [ ]:
torch.manual_seed(123)
train_loader = load_data_augmented_pretrained('/data/train', batch_size=128, shuffle=True, transform=pretrained_transform(), augmentation_transform = random_rotation_transform(), augment_prob = 0.3, num_workers=1)
model = VGG16Pretrained(dropout=dropout)
trainer = ModelTrainer(
    model=model,
    train_loader=train_loader,
    device=device,
    optimizer_type=optimizer_type, #to fill
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    log_file=f"vgg_pretrained.json",
    save_dir="./saved_models/vgg_pretrained/rotation",
    max_batches=None,
    valid_loader=valid_loader
)
trainer.train(10)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:03<00:00, 158MB/s]


is cuda True
Batch count 0
Batch count 10
Batch count 20
Batch count 30
Batch count 40
Batch count 50
Batch count 60
Batch count 70
Batch count 80
Batch count 90
Batch count 100
Batch count 110
Batch count 120
Batch count 130
Batch count 140
Batch count 150
Batch count 160
Batch count 170
Batch count 180
Batch count 190
Batch count 200
Batch count 210
Batch count 220
Batch count 230
Batch count 240
Batch count 250
Batch count 260
Batch count 270
Batch count 280
Batch count 290
Batch count 300
Batch count 310
Batch count 320
Batch count 330
Batch count 340
Batch count 350
Batch count 360
Batch count 370
Batch count 380
Batch count 390
Batch count 400
Batch count 410
Batch count 420
Batch count 430
Batch count 440
Batch count 450
Batch count 460
Batch count 470
Batch count 480
Batch count 490
Batch count 500
Batch count 510
Batch count 520
Batch count 530
Batch count 540
Batch count 550
Batch count 560
Batch count 570
Batch count 580
Batch count 590
Batch count 600
Batch count 610
Batch 

In [ ]:
torch.manual_seed(123)
train_loader = load_data_augmented('/data/train', batch_size=128, shuffle=True, transform=pretrained_transform(), augmentation_transform = horizontal_flip_transform_resized(), augment_prob = 0.3, num_workers=1)
model = VGG16Pretrained(dropout=dropout)
trainer = ModelTrainer(
    model=model,
    train_loader=train_loader,
    device=device,
    optimizer_type=optimizer_type, #to fill
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    log_file=f"vgg_pretrained.json",
    save_dir="./saved_models/vgg_pretrained/horizontal_flip",
    max_batches=None,
    valid_loader=valid_loader
)
trainer.train(10)

is cuda True
Batch count 0
Batch count 10
Batch count 20
Batch count 30
Batch count 40
Batch count 50
Batch count 60
Batch count 70
Batch count 80
Batch count 90
Batch count 100
Batch count 110
Batch count 120
Batch count 130
Batch count 140
Batch count 150
Batch count 160
Batch count 170
Batch count 180
Batch count 190
Batch count 200
Batch count 210
Batch count 220
Batch count 230
Batch count 240
Batch count 250
Batch count 260
Batch count 270
Batch count 280
Batch count 290
Batch count 300
Batch count 310
Batch count 320
Batch count 330
Batch count 340
Batch count 350
Batch count 360
Batch count 370
Batch count 380
Batch count 390
Batch count 400
Batch count 410
Batch count 420
Batch count 430
Batch count 440
Batch count 450
Batch count 460
Batch count 470
Batch count 480
Batch count 490
Batch count 500
Batch count 510
Batch count 520
Batch count 530
Batch count 540
Batch count 550
Batch count 560
Batch count 570
Batch count 580
Batch count 590
Batch count 600
Batch count 610
Batch 

In [ ]:
torch.manual_seed(123)
train_loader = load_data_augmented('/data/train', batch_size=128, shuffle=True, transform=color_jitter_transform_resized(), augmentation_transform = color_jitter_transform_resized(), augment_prob = 0.3, num_workers=1)
model = VGG16Pretrained(dropout=dropout)
trainer = ModelTrainer(
    model=model,
    train_loader=train_loader,
    device=device,
    optimizer_type=optimizer_type, #to fill
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    log_file=f"vgg_pretrained.json",
    save_dir="./saved_models/vgg_pretrained/color_jitter",
    max_batches=None,
    valid_loader=valid_loader
)
trainer.train(10)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


is cuda True
Batch count 0
Batch count 10
Batch count 20
Batch count 30
Batch count 40
Batch count 50
Batch count 60
Batch count 70
Batch count 80
Batch count 90
Batch count 100
Batch count 110
Batch count 120
Batch count 130
Batch count 140
Batch count 150
Batch count 160
Batch count 170
Batch count 180
Batch count 190
Batch count 200
Batch count 210
Batch count 220
Batch count 230
Batch count 240
Batch count 250
Batch count 260
Batch count 270
Batch count 280
Batch count 290
Batch count 300
Batch count 310
Batch count 320
Batch count 330
Batch count 340
Batch count 350
Batch count 360
Batch count 370
Batch count 380
Batch count 390
Batch count 400
Batch count 410
Batch count 420
Batch count 430
Batch count 440
Batch count 450
Batch count 460
Batch count 470
Batch count 480
Batch count 490
Batch count 500
Batch count 510
Batch count 520
Batch count 530
Batch count 540
Batch count 550
Batch count 560
Batch count 570
Batch count 580
Batch count 590
Batch count 600
Batch count 610
Batch 

In [ ]:
torch.manual_seed(123)
train_loader = load_data_augmented('/data/train', batch_size=128, shuffle=True, transform=combined_augmentation_transform_resized(), augmentation_transform = combined_augmentation_transform_resized(), augment_prob = 0.3, num_workers=1)
model = VGG16Pretrained(dropout=dropout)
trainer = ModelTrainer(
    model=model,
    train_loader=train_loader,
    device=device,
    optimizer_type=optimizer_type, #to fill
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    log_file=f"vgg_pretrained.json",
    save_dir="./saved_models/vgg_pretrained/combined",
    max_batches=None,
    valid_loader=valid_loader
)
trainer.train(10)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


is cuda True
Batch count 0
Batch count 10
Batch count 20
Batch count 30
Batch count 40
Batch count 50
Batch count 60
Batch count 70
Batch count 80
Batch count 90
Batch count 100
Batch count 110
Batch count 120
Batch count 130
Batch count 140
Batch count 150
Batch count 160
Batch count 170
Batch count 180
Batch count 190
Batch count 200
Batch count 210
Batch count 220
Batch count 230
Batch count 240
Batch count 250
Batch count 260
Batch count 270
Batch count 280
Batch count 290
Batch count 300
Batch count 310
Batch count 320
Batch count 330
Batch count 340
Batch count 350
Batch count 360
Batch count 370
Batch count 380
Batch count 390
Batch count 400
Batch count 410
Batch count 420
Batch count 430
Batch count 440
Batch count 450
Batch count 460
Batch count 470
Batch count 480
Batch count 490
Batch count 500
Batch count 510
Batch count 520
Batch count 530
Batch count 540
Batch count 550
Batch count 560
Batch count 570
Batch count 580
Batch count 590
Batch count 600
Batch count 610
Batch 

In [ ]:
torch.manual_seed(123)
train_loader = load_data_augmented('/data/train', batch_size=128, shuffle=True, transform=cutout_transform_resized(), augmentation_transform = cutout_transform_resized(), augment_prob = 0.3, num_workers=1)
model = VGG16Pretrained(dropout=dropout)
trainer = ModelTrainer(
    model=model,
    train_loader=train_loader,
    device=device,
    optimizer_type=optimizer_type, #to fill
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    log_file=f"vgg_pretrained.json",
    save_dir="./saved_models/vgg_pretrained/cutout",
    max_batches=None,
    valid_loader=valid_loader
)
trainer.train(10)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


is cuda True
Batch count 0
Batch count 10
Batch count 20
Batch count 30
Batch count 40
Batch count 50
Batch count 60
Batch count 70
Batch count 80
Batch count 90
Batch count 100
Batch count 110
Batch count 120
Batch count 130
Batch count 140
Batch count 150
Batch count 160
Batch count 170
Batch count 180
Batch count 190
Batch count 200
Batch count 210
Batch count 220
Batch count 230
Batch count 240
Batch count 250
Batch count 260
Batch count 270
Batch count 280
Batch count 290
Batch count 300
Batch count 310
Batch count 320
Batch count 330
Batch count 340
Batch count 350
Batch count 360
Batch count 370
Batch count 380
Batch count 390
Batch count 400
Batch count 410
Batch count 420
Batch count 430
Batch count 440
Batch count 450
Batch count 460
Batch count 470
Batch count 480
Batch count 490
Batch count 500
Batch count 510
Batch count 520
Batch count 530
Batch count 540
Batch count 550
Batch count 560
Batch count 570
Batch count 580
Batch count 590
Batch count 600
Batch count 610
Batch 

In [10]:
torch.manual_seed(123)
train_loader = load_data_augmented('/data/train', batch_size=128, shuffle=True, transform=pretrained_transform(), augmentation_transform = horizontal_flip_transform_resized(), augment_prob = 0.3, num_workers=1)
model = VGG16Pretrained(dropout=dropout)
trainer = ModelTrainer(
    model=model,
    train_loader=train_loader,
    device=device,
    optimizer_type=optimizer_type, #to fill
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    log_file=f"vgg_pretrained.json",
    save_dir="./saved_models/vgg_pretrained/horizontal_flip_20",
    max_batches=None,
    valid_loader=valid_loader
)
trainer.train(20, epochs_to_save=[5, 10, 15, 20])

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:05<00:00, 99.3MB/s]


is cuda True
Batch count 0
Batch count 10
Batch count 20
Batch count 30
Batch count 40
Batch count 50
Batch count 60
Batch count 70
Batch count 80
Batch count 90
Batch count 100
Batch count 110
Batch count 120
Batch count 130
Batch count 140
Batch count 150
Batch count 160
Batch count 170
Batch count 180
Batch count 190
Batch count 200
Batch count 210
Batch count 220
Batch count 230
Batch count 240
Batch count 250
Batch count 260
Batch count 270
Batch count 280
Batch count 290
Batch count 300
Batch count 310
Batch count 320
Batch count 330
Batch count 340
Batch count 350
Batch count 360
Batch count 370
Batch count 380
Batch count 390
Batch count 400
Batch count 410
Batch count 420
Batch count 430
Batch count 440
Batch count 450
Batch count 460
Batch count 470
Batch count 480
Batch count 490
Batch count 500
Batch count 510
Batch count 520
Batch count 530
Batch count 540
Batch count 550
Batch count 560
Batch count 570
Batch count 580
Batch count 590
Batch count 600
Batch count 610
Batch 